In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

### Installing beautifulsoup4 to explore the input data

In [5]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |           py36_1         143 KB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    certifi-2019.6.16          |           py36_0         154 KB  anaconda
    soupsieve-1.8              |           py36_0         104 KB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.8-py36_0        a

In [8]:
from bs4 import BeautifulSoup

### Download and parse the data

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [9]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [10]:
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

In [11]:
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

'''for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    print(row)  '''
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

toronto_data = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"]).replace('\n','', regex=True)
toronto_data=toronto_data.drop(toronto_data.index[[0]])
toronto_data.head()
#toronto_data.sort_values(["PostalCode"], axis=0,ascending=True, inplace=True)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### 288 rows fetched

In [12]:
toronto_data.shape

(288, 3)

### Ignore cells with a borough that is Not assigned.

In [13]:
toronto_data.drop(toronto_data[toronto_data['Borough'] == "Not assigned"].index, inplace = True)
toronto_data.shape

(211, 3)

### These two rows will be combined into one row with the neighborhoods separated with a comma

In [14]:
toronto_data_merge = toronto_data.groupby(['PostalCode','Borough'],as_index=False,sort=False).agg(','.join)

In [15]:
toronto_data_merge.head(25)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### filter rows having  Neighborhood as "Not assigned" and replace as like will column borough #ex:PostalCode M7A

In [16]:
#toronto_data_merge_del = toronto_data_merge[toronto_data_merge['Neighborhood']=='Not assigned']
#toronto_data_merge_del.shape #only record M7A	Queen's Park	Not assigned
#toronto_data_merge[toronto_data_merge['PostalCode']=='M7A']

In [17]:
toronto_data_merge.loc[toronto_data_merge['Neighborhood']=='Not assigned', 'Neighborhood'] = toronto_data_merge['Borough']
#toronto_data_merge[toronto_data_merge['PostalCode']=='M7A']
toronto_data_merge #ex:PostalCode M7A

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### use the .shape method to print the number of rows of your dataframe.

In [18]:
toronto_data_merge.shape

(103, 3)

In [2]:
conda install -c conda-forge geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

### Importing geocode data from http://cocl.us/Geospatial_data and creating Latitude and Longitude column

In [19]:
filename = "http://cocl.us/Geospatial_data"
headers = ["PostalCode","Latitude","Longitude"]
Geospatial_data = pd.read_csv(filename,names = headers)
#Geospatial_data = pd.read_csv(filename)
Geospatial_data=Geospatial_data.drop(Geospatial_data.index[[0]])
Geospatial_data.head()


,PostalCode,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174
5,M1H,43.773136,-79.2394761


In [20]:
#toronto_data_merge.sort_values(["PostalCode"], axis=0,ascending=True, inplace=True)
toronto_data_merge.shape #(103, 3)
Geospatial_data.shape #(103, 3)


(103, 3)

In [21]:
neighborhoods = pd.merge(toronto_data_merge, Geospatial_data, on ='PostalCode')
neighborhoods.shape
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6571618,-79.3789371


### Use geopy library to get the latitude and longitude values of Toronto City

In [22]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [23]:
neighborhoods.Latitude = neighborhoods.Latitude.astype(float)
neighborhoods.Longitude = neighborhoods.Longitude.astype(float)
neighborhoods.dtypes


PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

### Get unique values in column Borough

In [24]:
neighborhoods.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

#### Create a map of Toronto with neighborhoods superimposed on top.

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in "Central Toronto". So let's slice the original dataframe and create a new dataframe of the "Central Toronto" data.

In [27]:
CentralToronto_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
CentralToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Let's get the geographical coordinates of Central Toronto.

In [28]:
address = 'Central Toronto, Toronto canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


let's visualizat Central Toronto the neighborhoods in it.

In [29]:
# create map of Manhattan using latitude and longitude values
map_CentralToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(CentralToronto_data['Latitude'], CentralToronto_data['Longitude'], CentralToronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CentralToronto)  
    
map_CentralToronto

#### Define Foursquare Credentials and Version

In [30]:
CLIENT_ID = 'OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ' # your Foursquare ID
CLIENT_SECRET = 'C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ
CLIENT_SECRET:C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [33]:
CentralToronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [34]:
neighborhood_latitude = CentralToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CentralToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = CentralToronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [35]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ&client_secret=C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

Send the GET request and examine the resutls

In [37]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


And how many venues were returned by Foursquare?

In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Central Toronto*.

In [42]:
# type your answer here

CentralToronto_venues = getNearbyVenues(names=CentralToronto_data['Neighborhood'],
                                   latitudes=CentralToronto_data['Latitude'],
                                   longitudes=CentralToronto_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
North Toronto West
The Annex,North Midtown,Yorkville
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West


#### Let's check the size of the resulting dataframe

In [43]:
print(CentralToronto_venues.shape)
CentralToronto_venues.head()

(111, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Roselawn,43.711695,-79.416936,Menchie's St. Clair West,43.707664,-79.414301,Ice Cream Shop


Let's check how many venues were returned for each neighborhood

In [44]:
CentralToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,8,8,8,8,8,8
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",15,15,15,15,15,15
"Forest Hill North,Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park,Summerhill East",4,4,4,4,4,4
North Toronto West,18,18,18,18,18,18
Roselawn,2,2,2,2,2,2
"The Annex,North Midtown,Yorkville",24,24,24,24,24,24


#### Let's find out how many unique categories can be curated from all the returned venues

In [45]:
print('There are {} uniques categories.'.format(len(CentralToronto_venues['Venue Category'].unique())))

There are 59 uniques categories.


## 3. Analyze Each Neighborhood

In [46]:
# one hot encoding
CentralToronto_onehot = pd.get_dummies(CentralToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CentralToronto_onehot['Neighborhood'] = CentralToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CentralToronto_onehot.columns[-1]] + list(CentralToronto_onehot.columns[:-1])
CentralToronto_onehot = CentralToronto_onehot[fixed_columns]

CentralToronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,...,0,0,0,0,0
1,Lawrence Park,0,0,0,0,...,0,0,0,0,0
2,Lawrence Park,0,0,0,0,...,0,0,0,0,0
3,Roselawn,0,0,0,0,...,0,0,0,0,0
4,Roselawn,0,0,0,0,...,0,0,0,0,0


In [47]:
CentralToronto_onehot.shape

(111, 60)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
CentralToronto_grouped = CentralToronto_onehot.groupby('Neighborhood').mean().reset_index()
CentralToronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000,...,0.030303,0.00,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.125,...,0.000000,0.00,0.000000,0.000000,0.000000
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.066667,0.000000,0.066667,0.000,...,0.000000,0.00,0.000000,0.066667,0.000000
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000,...,0.000000,0.25,0.000000,0.000000,0.000000
4,Lawrence Park,0.000000,0.000000,0.000000,0.000,...,0.000000,0.00,0.000000,0.000000,0.000000
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000,...,0.000000,0.25,0.000000,0.000000,0.000000
6,North Toronto West,0.000000,0.000000,0.000000,0.000,...,0.000000,0.00,0.000000,0.000000,0.055556
7,Roselawn,0.000000,0.000000,0.000000,0.000,...,0.000000,0.00,0.000000,0.000000,0.000000
8,"The Annex,North Midtown,Yorkville",0.041667,0.041667,0.000000,0.000,...,0.000000,0.00,0.041667,0.000000,0.000000


#### Let's confirm the new size

In [49]:
CentralToronto_grouped.shape

(9, 60)

#### Let's print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in CentralToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CentralToronto_grouped[CentralToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1      Sandwich Place  0.09
2        Dessert Shop  0.09
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Davisville North----
               venue  freq
0  Food & Drink Shop  0.12
1              Hotel  0.12
2      Grocery Store  0.12
3                Gym  0.12
4     Clothing Store  0.12


----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
                 venue  freq
0          Coffee Shop  0.13
1                  Pub  0.13
2  American Restaurant  0.07
3  Fried Chicken Joint  0.07
4           Sports Bar  0.07


----Forest Hill North,Forest Hill West----
                venue  freq
0       Jewelry Store  0.25
1               Trail  0.25
2                Park  0.25
3    Sushi Restaurant  0.25
4  Salon / Barbershop  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  American Restaurant  0

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CentralToronto_grouped['Neighborhood']

for ind in np.arange(CentralToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CentralToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,...,Café,Coffee Shop,Brewery,Park,Greek Restaurant
1,Davisville North,Hotel,Breakfast Spot,Gym,Grocery Store,...,Clothing Store,Sandwich Place,Food & Drink Shop,Farmers Market,History Museum
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,American Restaurant,Supermarket,...,Liquor Store,Light Rail Station,Fried Chicken Joint,Sports Bar,Sushi Restaurant
3,"Forest Hill North,Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Park,...,Discount Store,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,...,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [53]:
# set number of clusters
kclusters = 5

CentralToronto_grouped_clustering = CentralToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CentralToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 3, 0, 2, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CentralToronto_merged = CentralToronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CentralToronto_merged = CentralToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CentralToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,...,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,...,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,...,Clothing Store,Sandwich Place,Food & Drink Shop,Farmers Market,History Museum
3,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,...,Discount Store,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,...,Park,Diner,Dessert Shop,Rental Car Location,Salon / Barbershop


Finally, let's visualize the resulting clusters

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CentralToronto_merged['Latitude'], CentralToronto_merged['Longitude'], CentralToronto_merged['Neighborhood'], CentralToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [56]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 0, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,0,Trail,Tennis Court,Park,...,Discount Store,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store


#### Cluster 2

In [57]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 1, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Ice Cream Shop,Garden,Discount Store,...,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant


#### Cluster 3

In [58]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 2, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,2,Hotel,Breakfast Spot,Gym,...,Clothing Store,Sandwich Place,Food & Drink Shop,Farmers Market,History Museum
4,Central Toronto,2,Sporting Goods Shop,Coffee Shop,Yoga Studio,...,Park,Diner,Dessert Shop,Rental Car Location,Salon / Barbershop
5,Central Toronto,2,Café,Sandwich Place,Coffee Shop,...,Cosmetics Shop,Grocery Store,Indian Restaurant,Jewish Restaurant,Liquor Store
6,Central Toronto,2,Pizza Place,Dessert Shop,Sandwich Place,...,Café,Coffee Shop,Brewery,Park,Greek Restaurant
8,Central Toronto,2,Pub,Coffee Shop,American Restaurant,...,Liquor Store,Light Rail Station,Fried Chicken Joint,Sports Bar,Sushi Restaurant


#### Cluster 4

In [59]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 3, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,3,Swim School,Bus Line,Park,...,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store


#### Cluster 5

In [60]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 4, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,4,Trail,Jewelry Store,Sushi Restaurant,...,Discount Store,Health & Beauty Service,Gym / Fitness Center,Gym,Grocery Store
